Valuing Actions by Estimating Probabilities

In [ ]:
import pandas as pd
import numpy as np
import joblib

import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")
from config import chain_file_path
from exp_vaep.exp_vaep_config import *

from exp_vaep.domain.preprocessing.data_preprocessor import ExpVAEPPreprocessor
from exp_vaep.domain.preprocessing.preprocessing import *
import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load Chain Data

In [ ]:
chains = pd.read_csv(chain_file_path)
chains.tail()

Preprocessing

In [ ]:
expected_scores_path_dict = {
    'set':{'goal':{'preprocessor':exp_goal_set_preprocessor_file_path,
                   'model':exp_goal_set_model_file_path},
           'behind':{'preprocessor':exp_behind_set_preprocessor_file_path,
                     'model':exp_behind_set_model_file_path},
           'miss':{'preprocessor':exp_miss_set_preprocessor_file_path,
                   'model':exp_miss_set_model_file_path}},
     'open':{'goal':{'preprocessor':exp_goal_open_preprocessor_file_path,
                    'model':exp_goal_open_model_file_path},
           'behind':{'preprocessor':exp_behind_open_preprocessor_file_path,
                     'model':exp_behind_open_model_file_path},
           'miss':{'preprocessor':exp_miss_open_preprocessor_file_path,
                   'model':exp_miss_open_model_file_path}}
}

In [ ]:
score_chains = get_expected_scores(chains, expected_scores_path_dict)
schema_chains = convert_chains_to_schema(score_chains)

In [ ]:
preproc = ExpVAEPPreprocessor(expected_scores_path_dict)

In [ ]:
preproc.fit(chains)

In [ ]:
exp_vaep_features = preproc.transform(chains)
exp_vaep_features.head()

In [ ]:
exp_vaep_labels = create_gamestate_labels(schema_chains)

In [ ]:
exp_vaep_modelling_data = pd.concat([schema_chains, exp_vaep_features, exp_vaep_labels], axis=1)
exp_vaep_modelling_data.head()

Train Test Split

In [ ]:
exp_vaep_modelling_data = get_stratified_train_test_val_columns(exp_vaep_modelling_data, response="exp_scores")
exp_vaep_modelling_data = get_stratified_train_test_val_columns(exp_vaep_modelling_data, response="exp_concedes")

In [ ]:
exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresTrainingSet']]['exp_scores'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresTestSet']]['exp_scores'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresValidationSet']]['exp_scores'].mean()

In [ ]:
exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesTrainingSet']]['exp_concedes'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesTestSet']]['exp_concedes'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesValidationSet']]['exp_concedes'].mean()

In [ ]:
exp_vaep_modelling_data.tail()

In [ ]:
exp_vaep_modelling_data.to_csv("../data/exp_vaep_modelling_data_v2.csv", index=False)

In [ ]:
joblib.dump(preproc, "../model_outputs/preprocessor/exp_vaep_preprocessor_v2.joblib")